In [10]:
import sys, os
sys.path.append(os.path.abspath('..'))
from cone_prog_refine import *
import numpy as np
import scipy.optimize

In [88]:
dim_dict = {'z': 2, 'l': 20,  'q': [30, 40], 's': [13, 14, 15], 'ep': 100, 'ed': 100}
A, b, c, _, x_true, s_true, y_true = generate_problem(dim_dict, mode='solvable')
cache = make_prod_cone_cache(dim_dict)
z_true = xsy2z(x_true, s_true, y_true)
residual(z_true, A, b, c, cache)
m,n = A.shape

In [89]:
fun = lambda z: np.linalg.norm(residual(z, A, b, c, cache) / z[-1] )**2

def lsqr_D(z, dz, A, b, c, cache, residual):
    return residual_D(z, dz, A, b, c, cache) / z[-1] - (residual /
                                                        z[-1]**2) * dz[-1]

def lsqr_DT(z, dres, A, b, c, cache, residual):
    m, n = A.shape
    e_minus1 = np.zeros(n + m + 1)
    e_minus1[-1] = 1.
    return residual_DT(z, dres, A, b, c, cache) / z[-1] - (dres@residual /
                                                           z[-1]**2) * e_minus1

def jac(z):
    res = residual(z, A, b, c, cache)
    return (lsqr_D(z, res, A, b, c, cache, res) + lsqr_DT(z, res, A, b, c, cache, res))

In [90]:
z_init = np.ones(n+m+1)
#z_init = z_true + np.random.randn(n+m+1) * 1E-1

In [91]:
result = scipy.optimize.fmin_l_bfgs_b(fun, z_init, fprime = jac, disp=101)

In [92]:
result

(array([ -3.86584532,   4.87070764,  -2.89125937, ...,  12.38940919,
         -1.88933318, 359.05229225]),
 0.7253739874488886,
 {'funcalls': 130,
  'grad': array([-0.00102058,  0.00342217, -0.00166726, ..., -0.02125222,
          0.02917141, -0.03199521]),
  'nit': 22,
  'task': b'ABNORMAL_TERMINATION_IN_LNSRCH',
  'warnflag': 2})

In [93]:
fun(z_init), fun(result[0])

(16529.539656676192, 0.725373987448884)

In [94]:
z_approx = result[0]
z_approx = refine(A, b, c, dim_dict, z_approx, iters=100)         



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
               Conic Solution Refinement              

it.    ||N(z)||_2       z[-1]   LSQR  btrks     time
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0	8.52e-01	1e+00	  0	0	0.01
1	7.11e-01	1e+00	  30	1	0.16
2	3.95e-01	1e+00	  30	0	0.31
3	3.95e-01	1e+00	  30	4	0.45
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Hit maximum number of backtracks.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


## Anderson 

In [108]:
N = lambda z: residual(z, A, b, c, cache)/np.abs(z[-1])

scipy.optimize.anderson(N, z_init, verbose=True, M = 20) 

0:  |F(x)| = 8.55623; step 1; tol 0.00398608
1:  |F(x)| = 8.02789; step 1; tol 0.792283
2:  |F(x)| = 4.42451; step 1; tol 0.564941
3:  |F(x)| = 3.35363; step 1; tol 0.517062
4:  |F(x)| = 2.58661; step 1; tol 0.535396
5:  |F(x)| = 2.08478; step 1; tol 0.584656
6:  |F(x)| = 1.73968; step 1; tol 0.626706
7:  |F(x)| = 1.49672; step 1; tol 0.666162
8:  |F(x)| = 1.32398; step 1; tol 0.704253
9:  |F(x)| = 1.20148; step 1; tol 0.741164
10:  |F(x)| = 1.11541; step 1; tol 0.775665
11:  |F(x)| = 1.06396; step 1; tol 0.818884
12:  |F(x)| = 1.03683; step 1; tol 0.854694
13:  |F(x)| = 1.02398; step 1; tol 0.877822
14:  |F(x)| = 1.01892; step 1; tol 0.891124
15:  |F(x)| = 1.01668; step 1; tol 0.896059
16:  |F(x)| = 1.01556; step 1; tol 0.898014
17:  |F(x)| = 1.0151; step 1; tol 0.899182
18:  |F(x)| = 1.01531; step 1; tol 0.900372
19:  |F(x)| = 1.01548; step 1; tol 0.900305
20:  |F(x)| = 1.01527; step 1; tol 0.899633
21:  |F(x)| = 1.01918; step 1; tol 0.906948
22:  |F(x)| = 1.01957; step 1; tol 0.9006

/Users/enzo/anaconda3/lib/python3.6/site-packages/scipy/optimize/nonlin.py:1121: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number3.707548e-17
  gamma = solve(self.a, df_f)
/Users/enzo/anaconda3/lib/python3.6/site-packages/scipy/optimize/nonlin.py:1121: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number3.209588e-17
  gamma = solve(self.a, df_f)
/Users/enzo/anaconda3/lib/python3.6/site-packages/scipy/optimize/nonlin.py:1121: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.098793e-26
  gamma = solve(self.a, df_f)


40:  |F(x)| = 12.711; step 1; tol 0.9


/Users/enzo/anaconda3/lib/python3.6/site-packages/scipy/optimize/nonlin.py:1121: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number4.781830e-28
  gamma = solve(self.a, df_f)


41:  |F(x)| = 12.711; step 1; tol 0.9


...

/Users/enzo/anaconda3/lib/python3.6/site-packages/scipy/optimize/nonlin.py:1121: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.283320e-36
  gamma = solve(self.a, df_f)


55:  |F(x)| = 12.711; step 1; tol 0.9
56:  |F(x)| = 12.711; step 0; tol 0.9


/Users/enzo/anaconda3/lib/python3.6/site-packages/scipy/optimize/nonlin.py:1121: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.323920e-36
  gamma = solve(self.a, df_f)


57:  |F(x)| = 12.711; step 1; tol 0.9
58:  |F(x)| = 12.711; step 1; tol 0.9
59:  |F(x)| = 12.711; step 1; tol 0.9
60:  |F(x)| = 12.711; step 1; tol 0.9
61:  |F(x)| = 12.711; step 1; tol 0.9
62:  |F(x)| = 12.711; step 1; tol 0.9
63:  |F(x)| = 12.711; step 1; tol 0.9
64:  |F(x)| = 12.711; step 1; tol 0.9
65:  |F(x)| = 12.711; step 1; tol 0.9
66:  |F(x)| = 12.711; step 1; tol 0.9
67:  |F(x)| = 12.711; step 1; tol 0.9
68:  |F(x)| = 12.711; step 1; tol 0.9
69:  |F(x)| = 12.711; step 1; tol 0.9
70:  |F(x)| = 12.711; step 1; tol 0.9
71:  |F(x)| = 12.711; step 1; tol 0.9
72:  |F(x)| = 12.711; step 1; tol 0.9
73:  |F(x)| = 12.711; step 1; tol 0.9
74:  |F(x)| = 12.711; step 1; tol 0.9
75:  |F(x)| = 12.711; step 1; tol 0.9
76:  |F(x)| = 12.711; step 1; tol 0.9
77:  |F(x)| = 12.711; step 1; tol 0.9
78:  |F(x)| = 12.711; step 1; tol 0.9
79:  |F(x)| = 12.711; step 1; tol 0.9
80:  |F(x)| = 12.711; step 1; tol 0.9
81:  |F(x)| = 12.711; step 1; tol 0.9
82:  |F(x)| = 12.711; step 1; tol 0.9
83:  |F(x)| 

KeyboardInterrupt: 

In [109]:
result = scipy.optimize.root(N, z_init, method='hybr') 

In [110]:
print(result)

    fjac: array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.81356889e-01],
       [ 8.32667268e-17,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  6.95715832e-02, -4.38859359e-01],
       [-7.63278329e-17,  5.55111512e-17,  0.00000000e+00, ...,
         0.00000000e+00,  1.40945680e-02,  2.63276330e-01],
       ...,
       [-2.22055929e-02,  5.26660585e-02,  6.73795252e-02, ...,
        -2.18680417e-10,  1.59232941e-10, -2.26719702e-11],
       [-1.57060602e-02,  1.66182180e-02, -1.90021354e-02, ...,
        -8.81049790e-17, -1.32175431e-10,  4.07927073e-11],
       [ 1.74819721e-02, -6.66696175e-02, -2.29721125e-03, ...,
        -2.74574200e-17,  3.64291930e-17, -4.74461269e-12]])
     fun: array([-1.72506282e-02, -2.53035800e-03, -1.82541867e-02, ...,
       -7.13123259e-01, -6.45373737e-01, -2.63991295e+01])
 message: 'The iteration is not making good progress, as measured by the \n  improvement from the las

In [111]:
np.linalg.norm(N(result['x']))

33.406614003222174